#### Use titanic dataset. Use the attributes: Class of the room, Sex, Age, number of siblings/spouses, number of parents/children, passenger fare and port of embarkation information. Compare the result of Naïve bayes Classifier with the Decision tree classifier and Logistic Regessor implemented in Lab Session 4.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/rahul96rajan/sample_datasets/master/titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.shape

(891, 12)

In [4]:
data.drop(labels=['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'], axis=1, inplace=True)

In [5]:
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [6]:
data.dropna(subset=['Embarked'], inplace=True)

In [7]:
X = data.drop(labels='Survived', axis=1)
y = data['Survived']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
X_train['Age'].value_counts()

30.00    22
24.00    22
18.00    21
25.00    20
28.00    18
         ..
12.00     1
32.50     1
70.50     1
20.50     1
0.42      1
Name: Age, Length: 84, dtype: int64

In [10]:
X_train.isna().sum()

Pclass        0
Sex           0
Age         129
SibSp         0
Parch         0
Embarked      0
dtype: int64

In [11]:
y_train[X_train['Age'].isnull()].sum() 
# i.e; 36 outof 129 poeple with missing age survived

36

In [12]:
print('Mode:({0}),  Median:{1},'
      '  Mean:{2}'.format(X_train['Age'].mode().values, X_train['Age'].median(),
                        X_train['Age'].mean()))

Mode:([24. 30.]),  Median:29.0,  Mean:29.754040968342643


In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

i_median = SimpleImputer(strategy='median')
i_median.fit(X_train['Age'].values.reshape(-1, 1))

leber = LabelEncoder()
leber.fit(X_train['Sex'].values.reshape(-1, 1).ravel())

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train[['Embarked']])

def preprocess(data):
    df = data.copy()
    df.loc[:, 'Age'] = i_median.transform(df['Age'].values.reshape(-1,1))
    df.loc[:, 'Sex'] = leber.transform(df['Sex'].values.reshape(-1, 1).ravel())
    ohed_pd = pd.DataFrame(enc.transform(df[['Embarked']]).toarray(),
                           columns=['Emb_S','Emb_Q','Emb_S'], index=df.index)
    df = df.join(ohed_pd)
    df['kins'] = df['SibSp'] + df['Parch']
    df.drop(labels=['Embarked','SibSp','Parch'], axis=1, inplace=True)
    return df


In [14]:
# Pre-Processing Training data
X_train = preprocess(X_train)

In [15]:
# Using GridSearch with LeaveOneOut Cross validation to get descent parameters.
from sklearn.naive_bayes import GaussianNB 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
gnb = GaussianNB()

params = {'var_smoothing':[1e-13, 1e-11, 1e-9, 1e-7, 1e-5]}

grid_search = GridSearchCV(gnb, params, cv=kf, scoring='f1',
                           return_train_score=True)
grid_search.fit(X_train, y_train)
grid_search.best_estimator_

GaussianNB(var_smoothing=1e-05)

In [16]:
clf = GaussianNB(var_smoothing=1e-05)
clf.fit(X_train, y_train)


X_test = preprocess(X_test)
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

In [17]:
y_test_pred = clf.predict(X_test)

In [20]:
from sklearn.metrics import f1_score

print('F1 score (train): ', f1_score(y_train, y_train_pred))
print('F1 score (test): ', f1_score(y_test, y_test_pred))

F1 score (train):  0.701688555347092
F1 score (test):  0.7419354838709677


In [21]:
from sklearn.metrics import accuracy_score

print('Accuracy (train): ', accuracy_score(y_train, y_train_pred))
print('Accuracy (test): ', accuracy_score(y_test, y_test_pred))

Accuracy (train):  0.7612612612612613
Accuracy (test):  0.7847533632286996


### <u>Comparison</u>: Accuracy in Logistic Regression(0.834) is higher than that of GaussianNB(0.7847) for titanic dataset.